# Limpieza de datos

La mayoría de la gente piensa que ser científico de datos significa estar corriendo modelos avanzados de Machine Learning todo el tiempo. La realidad es muy distinta:

![Tiempo de un científico de datos](./img/datascientist_time.jpeg)

La gran mayoría del tiempo se va **limpiando y organizando** los datos con los que queremos trabajar.

Los científicos de datos utilizamos herramientas como Pandas, Numpy, Matplotlib y Seaborn para limpiar los datos con los que queremos hacer algo.

## ETL

Un tipo de tarea que realizamos con gran frecuencia los científicos de datos son los **ETL**.

![Proceso de ETL](./img/etl.png)

ETL es un acrónimo que significa Extract, Transform, Load. Es un proceso que se utiliza para extraer datos de una fuente, transformarlos en un formato que sea adecuado para el análisis y cargarlos en una base de datos o algún otro sistema de almacenamiento.

## Ejemplo práctico

Los datos que usaremos para esta limpieza y nuestro siguiente análisis son datos de incidencia delictiva en nuestro país.

La iniciativa de datos abiertos del gobierno de México nos proporciona datos de incidencia delictiva desde 2015 hasta la fecha. Los datos se actualizan todos los meses y se pueden descargar desde el siguiente enlace: https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva

---

Como podemos ver en el portal, se proporcionan los datos tanto a nivel estatal como a nivel municipal. En este caso, utilizaremos los datos a nivel estatal.

Descarguemos los datos y guardemos el archivo CSV en la carpeta data con el nombre `datos_delitos.csv`.

---

Ahora leamos el archivo CSV y veamos cómo se ven los datos.

Primero que nada, importemos pandas

In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('data/datos_delitos.csv', encoding='latin1')


Aquí tenemos un error muy común que suele ocurrir cuando un archivo se guarda en una computadora con cierto "encoding". 

Un encoding es una tabla que relaciona un número con un carácter. Por ejemplo, en la tabla ASCII, el número 65 corresponde a la letra "A".

Si el archivo que estamos leyendo fue guardado con un encoding distinto al que pandas espera, nos arrojará un error. 

Para solucionar esto, podemos utilizar el parámetro `encoding` de la función `pd.read_csv()` y especificar el encoding correcto.

¿Pero cómo sabemos con qué encoding cuenta el archivo?

La realidad es que la gran mayoría de los archivos los encontrarán en encoding utf-8 y Pandas no va a dar ningún error. Aquí estamos teniendo este problema porque la computadora que utilizan para generar este archivo uso un encoding diferente a utf-8.

En México, por lo general, si un archivo no está en utf-8, lo más seguro es que esté en `ISO-8859-1` o `latin1`.

Intentemos cargar el archivo especificando el encoding `ISO-8859-1`.

In [ ]:
df = pd.read_csv('data/datos_delitos.csv', encoding='ISO-8859-1')
df.head()

Y vemos que ya podemos leer correctamente el archivo.


¿Existe alguna forma de verificar el encoding de un archivo sin tener que estar adivinando?

ChatGTP generó el siguiente código:

In [ ]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read()
    result = chardet.detect(rawdata)
    return result

file_path = './data/datos_delitos.csv'
encoding_info = detect_encoding(file_path)
print(f"Detected encoding: {encoding_info['encoding']}")

Sigamos...

In [ ]:
df.tail(3)

In [ ]:
df.info()

Bien. Están muy bien los datos. Sin embargo, tenemos un problema con el que es muy común encontrarnos. 

Resulta que el formato en el que está el archivo es fácil entender por seres humanos:


```markdown
| Año      | Entidad  | Enero    | Febrero  | Mes X    |
|----------|----------|----------|----------|----------|
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
|   ...    |   ...    |   ...    |   ...    |   ...    |
```

Vemos que tenemos los meses como encabezados. Es decir, el archivo está tratando cada mes como si fuera una variable. 

Nosotros como científicos de datos estamos más interesados en conjuntos de datos que no estén en este formato de "resumen" o "tabla dinámica". Para nosotros, lo ideal sería que cada mes fuera simplemente una observación más en nuestro conjunto de datos. Es decir, queremos transformar la tabla de arriba en:

```markdown
| Año      | Entidad  | Mes        |
|----------|----------|------------|
|   ...    |   ...    |   Enero    |
|   ...    |   ...    |   Febrero  |
|   ...    |   ...    |   Marzo    |
|   ...    |   ...    |   Abril    |
|   ...    |   ...    |   Mes X    |
```

A este tipo de formato le llamos "formato largo de datos". 

Haremos las siguientes limpiezas:
* Transformar los nombres de las columnas para que no tengan caracteres especiales y estén siempre en minúsculas
* Convertir el dataset a un formato de datos "largo"

In [6]:
for col in df.columns:
    print(col)

Año
Clave_Ent
Entidad
Bien jurídico afectado
Tipo de delito
Subtipo de delito
Modalidad
Enero
Febrero
Marzo
Abril
Mayo
Junio
Julio
Agosto
Septiembre
Octubre
Noviembre
Diciembre


In [7]:
len(df.columns)

19

### Limpiar nombres de columnas

In [8]:
def limpiar_columnas(df):
    columnas_limpias = []
    for col in df.columns:
        # convertir a minusculas, reemplazar espacios por guiones bajos y eliminar caracteres especiales
        col = col.lower().replace(" ", "_").replace("ñ", "ni").replace(".", "").replace("á", "a").replace("é", "e").replace("í","i").replace("ó", "o").replace("ú", "u")
        columnas_limpias.append(col)
    
    df.columns = columnas_limpias
    
    return df


In [9]:
df.head(2)

,Año,Clave_Ent,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
0,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,3,0,2,1,1,1,2.0,1.0,2.0,2.0,2.0,1.0
1,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1,1,0,0,0,1,0.0,1.0,0.0,0.0,0.0,1.0


In [10]:
df = limpiar_columnas(df)

In [11]:
df.head(3)

,anio,clave_ent,entidad,bien_juridico_afectado,tipo_de_delito,subtipo_de_delito,modalidad,enero,febrero,marzo,abril,mayo,junio,julio,agosto,septiembre,octubre,noviembre,diciembre
0,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma de fuego,3,0,2,1,1,1,2.0,1.0,2.0,2.0,2.0,1.0
1,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con arma blanca,1,1,0,0,0,1,0.0,1.0,0.0,0.0,0.0,1.0
2,2015,1,Aguascalientes,La vida y la Integridad corporal,Homicidio,Homicidio doloso,Con otro elemento,0,0,2,2,3,2,0.0,1.0,2.0,0.0,0.0,0.0


Muy bien. Ahora lo que queremos hacer es quitar algunas columnas. Nos interesan nada más las siguientes:

In [12]:
df[['anio', 'entidad']]

,anio,entidad
0,2015,Aguascalientes
1,2015,Aguascalientes
2,2015,Aguascalientes
3,2015,Aguascalientes
4,2015,Aguascalientes
...,...,...
31355,2024,Zacatecas
31356,2024,Zacatecas
31357,2024,Zacatecas
31358,2024,Zacatecas


In [13]:
df = df[['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'subtipo_de_delito', 'modalidad','enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']]
df.head(2)

,anio,clave_ent,entidad,tipo_de_delito,subtipo_de_delito,modalidad,enero,febrero,marzo,abril,mayo,junio,julio,agosto,septiembre,octubre,noviembre,diciembre
0,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con arma de fuego,3,0,2,1,1,1,2.0,1.0,2.0,2.0,2.0,1.0
1,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con arma blanca,1,1,0,0,0,1,0.0,1.0,0.0,0.0,0.0,1.0


### Formato largo de datos

Ahora, usaremos el método `melt` para convertir las columnas a observaciones.

Queremos convervar las coumnas:
* anio
* clave_ent
* entidad
* tipo_de_delito
* subtipo_de_delito
* modalidad

El resto de las columnas las vamos a juntar en una nueva columna llamada "nombre_mes" y sus valores los vamos a sumar en otra llamada "frecuencia"

In [14]:
print("Shape ", df.shape)

Shape  (31360, 18)


In [16]:
datos_long = df.melt(id_vars=['anio', 'clave_ent', 'entidad','tipo_de_delito', 'subtipo_de_delito', 'modalidad'], var_name='nombre_mes', value_name='frecuencia')

In [17]:
print("Shape: ", datos_long.shape)

Shape:  (376320, 8)


In [18]:
datos_long.head(5)

,anio,clave_ent,entidad,tipo_de_delito,subtipo_de_delito,modalidad,nombre_mes,frecuencia
0,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con arma de fuego,enero,3.0
1,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con arma blanca,enero,1.0
2,2015,1,Aguascalientes,Homicidio,Homicidio doloso,Con otro elemento,enero,0.0
3,2015,1,Aguascalientes,Homicidio,Homicidio doloso,No especificado,enero,2.0
4,2015,1,Aguascalientes,Homicidio,Homicidio culposo,Con arma de fuego,enero,0.0


In [19]:
datos_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376320 entries, 0 to 376319
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   anio               376320 non-null  int64  
 1   clave_ent          376320 non-null  int64  
 2   entidad            376320 non-null  object 
 3   tipo_de_delito     376320 non-null  object 
 4   subtipo_de_delito  376320 non-null  object 
 5   modalidad          376320 non-null  object 
 6   nombre_mes         376320 non-null  object 
 7   frecuencia         357504 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 23.0+ MB


Supongamos que para este análisis, no nos importan los niveles subtipo de delito y modalidad. O sea, no queremos tener la distinción entre homicidios dolosos y culposos (sé que son bastante diferentes, pero simplifiquemos nuestro ejemplo).

Vamos a agrupar nuestro dataframe por anio, clave_ent, entidad, tipo_de_delito y nombre_mes. Esto hará que todos los tipos de homicidios se sumen al tipo "homicidio" o todos los tipos de robo de vehículo (con o sin violencia) se sumen a "robo de vehículo".

In [21]:
datos_long = datos_long.groupby(['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'nombre_mes'])['frecuencia'].sum().reset_index()

In [22]:
datos_long[datos_long.tipo_de_delito == 'Robo'].sample(15)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
77666,2020,2,Baja California,Robo,diciembre,2324.0
17188,2016,4,Campeche,Robo,febrero,86.0
122787,2022,32,Zacatecas,Robo,enero,494.0
76711,2019,32,Zacatecas,Robo,marzo,570.0
129994,2023,15,México,Robo,octubre,11186.0
81029,2020,9,Ciudad de México,Robo,julio,6234.0
67593,2019,13,Hidalgo,Robo,noviembre,1195.0
144875,2024,14,Jalisco,Robo,septiembre,0.0
116548,2022,19,Nuevo León,Robo,febrero,1043.0
18149,2016,6,Colima,Robo,julio,637.0


Mostremos todos los estados y su respectiva clave

In [23]:
datos_long[['clave_ent', 'entidad']].drop_duplicates().sort_values('entidad')

,clave_ent,entidad
0,1,Aguascalientes
480,2,Baja California
960,3,Baja California Sur
1440,4,Campeche
2880,7,Chiapas
3360,8,Chihuahua
3840,9,Ciudad de México
1920,5,Coahuila de Zaragoza
2400,6,Colima
4320,10,Durango


Ahora Veamos todos los datos de delitos de una entidad en específico. Por ejemplo, Nuevo león

In [24]:
datos_long[datos_long['clave_ent'] == 19].sample(15)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
54809,2018,19,Nuevo León,Corrupción de menores,julio,18.0
85521,2020,19,Nuevo León,Contra el medio ambiente,noviembre,0.0
8759,2015,19,Nuevo León,Delitos cometidos por servidores públicos,septiembre,128.0
131782,2023,19,Nuevo León,Incumplimiento de obligaciones de asistencia f...,octubre,55.0
116160,2022,19,Nuevo León,Aborto,abril,9.0
101071,2021,19,Nuevo León,Lesiones,marzo,428.0
9058,2015,19,Nuevo León,Trata de personas,octubre,0.0
100977,2021,19,Nuevo León,Falsedad,noviembre,23.0
70401,2019,19,Nuevo León,Otros delitos contra la sociedad,noviembre,1.0
8917,2015,19,Nuevo León,Narcomenudeo,agosto,104.0


In [25]:
datos_long[(datos_long['clave_ent'] > 19) &  (datos_long['clave_ent'] < 24) & (datos_long['tipo_de_delito'] == 'Homicidio')].sample(15)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
72222,2019,23,Quintana Roo,Homicidio,junio,122.0
70787,2019,20,Oaxaca,Homicidio,septiembre,129.0
9336,2015,20,Oaxaca,Homicidio,abril,0.0
24704,2016,20,Oaxaca,Homicidio,mayo,136.0
87580,2020,23,Quintana Roo,Homicidio,febrero,131.0
132706,2023,21,Puebla,Homicidio,octubre,82.0
41024,2017,22,Querétaro,Homicidio,mayo,36.0
56862,2018,23,Quintana Roo,Homicidio,junio,71.0
148059,2024,21,Puebla,Homicidio,enero,86.0
41500,2017,23,Quintana Roo,Homicidio,febrero,25.0


### Valores de fechas

Finalmente, queremos tener una columna "fecha". Actualmente tenemos el año y el nombre del mes, pero no tenemos como tal una columna que tenga un tipo de dato fecha. Eso hace que filtrar por fecha sea complicado.

Por ejemplo, si queremos conocer todos los homicidios de Oaxaca en enero 2024, haríamos lo siguiente:

In [26]:
datos_long[
    (datos_long['clave_ent'] == 20) &
    (datos_long['tipo_de_delito'] == 'Homicidio') &
    (datos_long['anio'] == 2024) &
    (datos_long['nombre_mes'] == 'enero') 
]

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
147579,2024,20,Oaxaca,Homicidio,enero,157.0


Creemos una columna de fecha. 

Primero tenemos que convertir el nombre de mes a un número, en donde 1 es enero, 2 febrero, etc.

In [27]:
datos_long.sample(2)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia
27764,2016,26,Sonora,Secuestro,mayo,0.0
92855,2021,2,Baja California,Fraude,septiembre,203.0


In [28]:
datos_long["nueva_columna"] = "dato vacío"
datos_long.sample(6)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,nueva_columna
49290,2018,7,Chiapas,Otros delitos del Fuero Común,junio,103.0,dato vacío
90212,2020,28,Tamaulipas,Violación simple,mayo,36.0,dato vacío
130100,2023,16,Michoacán de Ocampo,Abuso de confianza,mayo,56.0,dato vacío
96170,2021,9,Ciudad de México,Falsedad,diciembre,42.0,dato vacío
11453,2015,24,San Luis Potosí,Trata de personas,julio,0.0,dato vacío
42931,2017,26,Sonora,Fraude,marzo,90.0,dato vacío


In [29]:
# Diccionario de ayuda para convertir
meses = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12
}

datos_long['mes'] = datos_long['nombre_mes'].map(meses)
datos_long.sample(5)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,nueva_columna,mes
34247,2017,8,Chihuahua,Extorsión,septiembre,1.0,dato vacío,9
108816,2022,3,Baja California Sur,Otros delitos que atentan contra la libertad p...,abril,7.0,dato vacío,4
115020,2022,16,Michoacán de Ocampo,Otros delitos contra la familia,abril,0.0,dato vacío,4
101367,2021,20,Oaxaca,Corrupción de menores,enero,1.0,dato vacío,1
52774,2018,14,Jalisco,Violación simple,octubre,34.0,dato vacío,10


In [30]:
datos_long["frecuencia_mas_10"] = datos_long["frecuencia"] + 10
datos_long.sample(5)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,nueva_columna,mes,frecuencia_mas_10
119603,2022,26,Sonora,Contra el medio ambiente,septiembre,0.0,dato vacío,9,10.0
72373,2019,23,Quintana Roo,Rapto,agosto,0.0,dato vacío,8,10.0
3162,2015,7,Chiapas,Narcomenudeo,junio,49.0,dato vacío,6,59.0
96284,2021,9,Ciudad de México,Narcomenudeo,mayo,438.0,dato vacío,5,448.0
125336,2023,6,Colima,Allanamiento de morada,mayo,16.0,dato vacío,5,26.0


In [33]:
datos_long["anio_mes"] = datos_long["anio"].astype(str) + datos_long["mes"].astype(str)
datos_long.sample(6)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,nueva_columna,mes,frecuencia_mas_10,anio_mes
54553,2018,18,Nayarit,Otros delitos contra la sociedad,agosto,1.0,dato vacío,8,11.0,20188
40504,2017,21,Puebla,Falsificación,febrero,7.0,dato vacío,2,17.0,20172
84906,2020,17,Morelos,Tráfico de menores,junio,0.0,dato vacío,6,10.0,20206
102748,2021,23,Quintana Roo,Abuso sexual,febrero,63.0,dato vacío,2,73.0,20212
32053,2017,3,Baja California Sur,Rapto,agosto,0.0,dato vacío,8,10.0,20178
37431,2017,14,Jalisco,Violencia familiar,enero,733.0,dato vacío,1,743.0,20171


In [34]:
# yyyy-mm-dd, yy-mm-dd, yymmdd, yyyy/dd/mm

In [35]:
# Agregamos la columna de fecha juntando el año y el mes
datos_long['fecha'] = pd.to_datetime(datos_long['anio'].astype(str) + datos_long['mes'].astype(str), format='%Y%m')
datos_long.sample(5)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,nueva_columna,mes,frecuencia_mas_10,anio_mes,fecha
110223,2022,6,Colima,Otros delitos contra la familia,enero,0.0,dato vacío,1,10.0,20221,2022-01-01
118055,2022,22,Querétaro,Violación simple,septiembre,31.0,dato vacío,9,41.0,20229,2022-09-01
98289,2021,13,Hidalgo,Otros delitos que atentan contra la vida y la ...,noviembre,29.0,dato vacío,11,39.0,202111,2021-11-01
94853,2021,6,Colima,Otros delitos contra el patrimonio,julio,29.0,dato vacío,7,39.0,20217,2021-07-01
90062,2020,28,Tamaulipas,Otros delitos contra la familia,diciembre,57.0,dato vacío,12,67.0,202012,2020-12-01


In [36]:
# Eliminamos las columnas que ya no necesitamos
datos_long = datos_long.drop(columns=['nueva_columna'])
datos_long.sample(5)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,mes,frecuencia_mas_10,anio_mes,fecha
62587,2019,3,Baja California Sur,Falsificación,marzo,4.0,3,14.0,20193,2019-03-01
105555,2021,28,Tamaulipas,Violación equiparada,enero,0.0,1,10.0,20211,2021-01-01
3756,2015,8,Chihuahua,Secuestro,abril,0.0,4,10.0,20154,2015-04-01
126480,2023,8,Chihuahua,Incesto,abril,0.0,4,10.0,20234,2023-04-01
49123,2018,7,Chiapas,Extorsión,marzo,19.0,3,29.0,20183,2018-03-01


Veamos los homicidios en oaxaca de enero 2024 a la fecha

In [38]:
datos_long[
    (datos_long.tipo_de_delito == "Homicidio") &
    (datos_long.clave_ent == 20) &
    (datos_long.fecha >= '2024-01-01')
].sort_values('fecha')

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,frecuencia,mes,frecuencia_mas_10,anio_mes,fecha
147579,2024,20,Oaxaca,Homicidio,enero,157.0,1,167.0,20241,2024-01-01
147580,2024,20,Oaxaca,Homicidio,febrero,170.0,2,180.0,20242,2024-02-01
147583,2024,20,Oaxaca,Homicidio,marzo,190.0,3,200.0,20243,2024-03-01
147576,2024,20,Oaxaca,Homicidio,abril,208.0,4,218.0,20244,2024-04-01
147584,2024,20,Oaxaca,Homicidio,mayo,236.0,5,246.0,20245,2024-05-01
147582,2024,20,Oaxaca,Homicidio,junio,202.0,6,212.0,20246,2024-06-01
147581,2024,20,Oaxaca,Homicidio,julio,0.0,7,10.0,20247,2024-07-01
147577,2024,20,Oaxaca,Homicidio,agosto,0.0,8,10.0,20248,2024-08-01
147587,2024,20,Oaxaca,Homicidio,septiembre,0.0,9,10.0,20249,2024-09-01
147586,2024,20,Oaxaca,Homicidio,octubre,0.0,10,10.0,202410,2024-10-01


In [39]:
datos_finales = datos_long[['anio', 'clave_ent', 'entidad', 'tipo_de_delito', 'nombre_mes', 'fecha', 'frecuencia']]
datos_finales.head(2)

,anio,clave_ent,entidad,tipo_de_delito,nombre_mes,fecha,frecuencia
0,2015,1,Aguascalientes,Aborto,abril,2015-04-01,0.0
1,2015,1,Aguascalientes,Aborto,agosto,2015-08-01,0.0


Ya que tenemos muestros datos bien estructurados, los podemos guardar en nuestra computadora. Los guardaremos con el nombre "delitos.csv"

In [40]:
datos_finales.to_csv('data/delitos.csv', index=False)